# Rangelands dynamics with Sentinel-1

This notebook gives an example of using Sentinel-1 analysis ready data (ARD) radar backscatter (Gamma nought) and dual-polarimetric decomposition (Alpha, Entropy and Anisotropy) for exploring annual rangeland dynamics.

Sentinel-1 (in this case it is the dual-pol Radar Vegetation Index (RVI) and Entropy bands) and Sentinel-2 NDVI are compared for multitemporal trends due to their relationships with vegetation biomass. A grasslands mask (read in as a shape file) is then applied to extract only the areas of interest, before the monthly means are generated for the RVI and Entropy bands. These monthly means as well as the annual range in RVI and Entropy are then output as GeoTIFF files.

The following steps are used:
1.  Load Sentinel-1 radar backscatter data through the datacube API
2.  Apply speckle filtering
3.  Calculate the dual-pol Radar Vegetation Index (RVI)
4.  Load Sentinel-1 dual-pol decomposition data
5.  Load Sentinel-2 data and calculate NDVI
6.  Interactively compare RVI, Entropy and NDVI time series
7.  Show annual statistics
8.  Generate monthly means
9.  Save results to GeoTIFF files

This notebook was written using the Virtual Desktop Infrastructure (VDI) on the National Computational Infrastructure (NCI), allowing access to the current Sentinel-1 and Digitial Earth Australia (DEA) ARD datasets. For this notebook to work you must first load the relevant modules:
-  module use /g/data/v10/public/modules/modulefiles
-  module load dea

# 1. Load Sentinel-1 radar backscatter data through the datacube API

Area of interest is in the Fitzroy Catchment of Western Australia

In [ ]:
# Import relevant modules for this Jupyter Notebook and start datacube
import sys
import datacube
import numpy as np
import xarray as xr
import geoviews as gv
import geoviews.feature as gf
from cartopy import crs
from datacube.storage import masking
from datacube.storage.masking import mask_to_dict
from datacube.utils import geometry
from datacube.utils.geometry import CRS
from matplotlib import pyplot as plt
from IPython.display import display
from IPython.display import HTML
import ipywidgets as widgets
from ipywidgets import interact
#from matplotlib import pyplot as plt
import matplotlib.image as mplImage
import warnings
warnings.filterwarnings('ignore', module='datacube')

import logging
logging.basicConfig(stream=sys.stdout, level=logging.CRITICAL)
for logger_name in ('boto', 'boto3', 'botocore', 's3transfer', 'rasterio', 'distributed'):
    logging.getLogger(logger_name).setLevel(logging.CRITICAL)
    
dc = datacube.Datacube(env='datacube')#, config='radar.conf')

In [ ]:
import dask
from dask_kubernetes import KubeCluster
from dask.distributed import Client,LocalCluster
from dask.distributed import wait, progress
cluster = None
# Click on the 'Dashboard link' to monitor calculation progress
cluster = LocalCluster()
cluster.scale_up(4)
#cluster.adapt(minimum=1, maximum=3, scale_factor=2, startup_cost="10s", wait_count=5, target_duration="10s")

In [ ]:
import distributed
client = distributed.Client(cluster)

In [ ]:
client

# Select area of interest for year of interest 
>  Currently only use 2017 or 2018 for full years <br>
>  Note that the area of interest must be small to avoid memory issues

In [ ]:
from ipyleaflet import Map, WMSLayer, LayersControl

m = Map(center=(-35.283, 149.128), zoom=12, layout=dict(height='600px'))
m

In [ ]:
Year = '2019'
myloc = 'ACT'
# Set up centre of area to analyse, and a buffer in metres around this centrepoint
lat, lon, buffer_m = -35.283, 149.128, (10*1000)
x, y = geometry.point(lon, lat, CRS('WGS84')).to_crs(CRS('EPSG:3577')).points[0]
query = {
'y': (y - buffer_m, y + buffer_m),
'x': (x - buffer_m, x + buffer_m),
'time': (Year + '-05-01', Year + '-07-31'),
'crs': 'EPSG:3577',
'output_crs': 'EPSG: 3577',
'resolution': (20, 20)
}

# Read radar backscatter, clean and smooth data

In [ ]:
bs=dc.load(product='s1_intensity_scene', group_by='solar_day', **query, dask_chunks={'time': 1})

# Identify images that have minimum nulls
# Uses code from https://github.com/fangfy/dea-projects/blob/master/water_interoperability/sentinel1_load_and_classify_nci.ipynb

total_px=len(bs.x)*len(bs.y)
valid=bs.where(bs.vv!=0).where(bs.vh!=0).count(dim=('x','y'))

good=(valid.vh/total_px)>0.5

bs_good = bs.sel(time=good)
# replace 0 with nan
bs_clean = bs_good.where(bs_good!=0)

# Adapted from https://stackoverflow.com/questions/39785970/speckle-lee-filter-in-python
from scipy.ndimage.filters import uniform_filter
from scipy.ndimage.measurements import variance

def lee_filter(da, size):
    img = da.values
    img_mean = uniform_filter(img, (size, size))
    img_sqr_mean = uniform_filter(img**2, (size, size))
    img_variance = img_sqr_mean - img_mean**2

    overall_variance = variance(img)

    img_weights = img_variance / (img_variance + overall_variance)
    img_output = img_mean + img_weights * (img - img_mean)
    return img_output

# save the nodata mask
nodata_mask = bs_clean.isnull().to_array().any(axis=0)
# Convert backscatter nans to 0 for lee filter
bs_good_zerofilled = bs_good.where(~bs_good.isnull(), 0)

# Apply speckle filter
smoothed_vv=bs_good_zerofilled.vv.groupby('time').apply(lee_filter, size=7)
smoothed_vh=bs_good_zerofilled.vh.groupby('time').apply(lee_filter, size=7)

# Create smoothed dataset with Nans and assign attributes
smoothed=smoothed_vv.to_dataset(name='vv')
smoothed['vh']=smoothed_vh
smoothed=smoothed.where(~nodata_mask)

# Remove unused data
bs_attrs = bs.attrs
smoothed = smoothed.assign_attrs(bs_attrs)
#smoothed = smoothed.persist()

del bs, bs_good, bs_clean
print('Backscatter data for ', myloc,': ',smoothed)

## View smoothed images (VV or VH)

In [ ]:
gv.extension('bokeh')
dataset = smoothed.load()
dataset_vv = gv.Dataset(dataset.drop('vh'), ['x', 'y', 'time'], 'vv', crs=crs.epsg(3577))
dataset_vv = dataset_vv.redim.range(vv=(0.0,0.2))
dataset_vh = gv.Dataset(sataset.drop('vv'), ['x', 'y', 'time'], 'vh', crs=crs.epsg(3577))
dataset_vh = dataset_vh.redim.range(vh=(0.0,0.08))

images = dataset_vv.to(gv.Image).opts(cmap='viridis', colorbar=True, projection=crs.epsg(3577), width=492, height=500) + dataset_vh.to(gv.Image).opts(colorbar=True, projection=crs.epsg(3577), width=492, height=500)

images

In [ ]:
# View smoothed data (Note - you might need to edit 'figsize' to improve appearance)
ntimes=len(smoothed.time.values)

@interact(mypol=['VV','VH'])

def plot(mypol='VV'):
    if mypol == 'VV': smoothed.vv.isel(time=slice(0,ntimes,2)).plot(col='time',col_wrap=4, vmin=0, vmax=0.2, figsize=(40,20));
    if mypol == 'VH': smoothed.vh.isel(time=slice(0,ntimes,2)).plot(col='time',col_wrap=4, vmin=0, vmax=0.08, figsize=(40,20));


## Calculate and view modified dual-pol Radar Vegetation Index images

In [ ]:
# View RVI data

smoothed['RVI'] = 4*smoothed.vh/(smoothed.vv + smoothed.vh)

ntimes=len(smoothed.time.values)
smoothed.RVI.isel(time=slice(0,ntimes,2)).plot(col='time',col_wrap=4, vmin=0.2, vmax=1.5, figsize=(20,10));

# Read in dual-pol decomposition data
## This results in Alpha, Anisotropy and Entropy bands

In [ ]:
dp=dc.load(product='s1_decomposition_scene', group_by='solar_day', **query, dask_chunks={'time': 1} )

dp = dp.where(dp.entropy!=0)
dp = dp.persist()
print('Dual polarimetric decomposition data for ', myloc,': ',dp)

## View Alpha or Entropy images

In [ ]:
ntimes=len(dp.time.values)

# View entropy (values range from 0-1.0) data 
# (can also change to look at 'anisotropy' (values ~0-1.0))

@interact(mypol=['Alpha','Entropy'])

def plot(mypol='Alpha'):
    if mypol == 'Alpha': dp.alpha.isel(time=slice(0,ntimes,2)).plot(col='time',col_wrap=4, vmin=40, vmax=80.0, figsize=(20,10));
    if mypol == 'Entropy': dp.entropy.isel(time=slice(0,ntimes,2)).plot(col='time',col_wrap=4, vmin=0, vmax=1.0, figsize=(20,10));


# Load Sentinel-2 to extract NDVI time series

In [ ]:
Year = '2019'
myloc = 'ACT'
# Set up centre of area to analyse, and a buffer in metres around this centrepoint
lat, lon, buffer_m = -35.283, 149.128, (20*1000)
x, y = geometry.point(lon, lat, CRS('WGS84')).to_crs(CRS('EPSG:3577')).points[0]
query = {
'y': (y - buffer_m, y + buffer_m),
'x': (x - buffer_m, x + buffer_m),
'time': (Year + '-05-16', Year + '-10-30'),
'crs': 'EPSG:3577',
'output_crs': 'EPSG: 3577',
'resolution': (-20, 20)
}


In [ ]:
dc_S2 = datacube.Datacube(env='datacube', app='dc-S2-extract')

# Read S2 green, nir and cloud mask bands for S2a and S2b
bands_of_interest = ['B04_20m', 'B03_20m','B02_20m', 'B8A_20m', 'SCL_20m']
s2_orig = dc_S2.load(product = 's2_l2a_scene', group_by='solar_day', measurements = bands_of_interest, **query, dask_chunks={'time': 1})

In [ ]:
s2 = s2_orig.rename({'B04_20m': 'red',
                     'B03_20m': 'green',
                     'B02_20m': 'blue',
                     'B8A_20m': 'nir',
                     'SCL_20m': 'SCL'})
del s2_orig


In [ ]:
from datacube.storage import masking
# Set all nodata pixels to `NaN`:
s2 = masking.mask_invalid_data(s2)

# Identify images that have minimum nulls and remove them
# Uses code from https://github.com/fangfy/dea-projects/blob/master/water_interoperability/sentinel1_load_and_classify_nci.ipynb

total_px=len(s2.x)*len(s2.y)
valid=s2.where(s2.red!=0).where(s2.SCL<=7).where(s2.SCL>=2).where(s2.SCL!=3).count(dim=('x','y'))

good=(valid.red/total_px)>0.5

s2_good = s2.sel(time=good)

# replace 0 with nan
s2_good_clean = s2_good.where(s2_good!=0)
del s2_good

s2 = s2_good_clean
del s2_good_clean


In [ ]:
s2 = s2.persist()

In [ ]:
image_array = s2[['red', 'green', 'blue']].to_array().compute()
percentile_stretch=(0.02, 0.95)
vmin, vmax = image_array.quantile(percentile_stretch).values
# Show the image
image_array.plot.imshow(col='time', col_wrap=4,vmin=vmin, vmax=vmax,robust=True)

In [ ]:
#s2_good_temp = s2_good.where(s2_good.fmask<2) # to remove cloud/shadow/nulls based on fmask
#s2_good_clean = s2_good_temp.where(s2_good_temp.fmask!=0)

#s2_good_clean = s2_good_clean.assign_attrs(bs_attrs)

#del s2_good

# Create NDVI band
s2['ndvi']=(s2.nir.astype(float) - s2.red.astype(float))/(s2.nir.astype(float) + s2.red.astype(float))

In [ ]:
# Remove any remaining erroneous values (where NDVI <-1.0 and NDVI > 1.0)
s2['ndvi'] = s2.ndvi.where(s2.ndvi > -1.0).where(s2.ndvi < 1.0)
print('Sentinel-2 data for ', myloc,': ',s2)

## View NDVI images

In [ ]:
# View S2 NDVI time series
ntimes=len(s2.ndvi.time.values)
s2.ndvi.plot(cmap='viridis', col='time', col_wrap=2,figsize=(20,20));

# Select area of interest to interactively to look at RVI, Entropy and NDVI time series

In [ ]:
# This is disabled as it won't work with jupyterlabs at the moment
#warnings.filterwarnings('ignore')

#def onclick(event):
#    global pixelx, pixely
#    x, y = int(event.xdata), int(event.ydata)
#    image_coords = smoothed.affine * (x, y)
#    pixelx = x 
#    pixely = y 
#    w.value = 'pixelx : {}, pixely : {}'.format(pixelx, pixely)
#    print('data values:',x,y)
#
#print('\033[1m' + 'Click on the pixel to view its time series below' + '\033[0m')
#
#fig = plt.figure(figsize = (5,5))
#plt.imshow(np.flip(smoothed.vv.mean(dim='time')), interpolation = 'nearest', clim=(0,0.1), extent=[smoothed.coords['x'].min(), smoothed.coords['x'].max(), smoothed.coords['y'].min(), smoothed.coords['y'].max()]); 
#
#w = widgets.HTML("Click on the pixel to view its time series below")
#
#cid = fig.canvas.mpl_connect('button_press_event', onclick)
#display(w)

## Plot monthly time series based on selected pixel

In [ ]:
pixelx, pixely = 1555010,-3956214
# plot RVI, Entropy and NDVI through time based on the x,y point selected

import ipywidgets as widgets
from ipywidgets import interact, interact_manual, interactive
warnings.filterwarnings('ignore', module='datacube')

# expand selected x,y pixel to make a square area of interest
xp, yp=slice(pixelx-100,pixelx+100), slice(pixely-100,pixely+100)

@interact(myname=['monthly RVI','monthly Entropy'])

def plot(myname='monthly RVI'):
    global myplot, myimage, mytitle
    if myname == 'monthly RVI': 
        (smoothed.sel(x=xp,y=yp).RVI.groupby('time.month').mean()).plot(color='r', figsize=(10,3));
        myplot = (smoothed.sel(x=xp,y=yp).RVI.groupby('time').mean())
        myimage = smoothed.RVI
        mytitle = 'RVI (all images)'
    if myname == 'monthly Entropy': 
        (dp.sel(x=xp,y=yp).entropy.groupby('time.month').mean()).plot(color='b', figsize=(10,3));
        myplot = (dp.sel(x=xp,y=yp).entropy.groupby('time').mean())
        myimage = dp.entropy
        mytitle = 'Entropy (all images)'

In [ ]:
print(s2.sel(x=xp,y=yp))

## View time series of selected timeseries (RVI or Entropy) based on all scenes
### Select a point to view scene

In [ ]:
# Click on a point in the time-series from RVI or Entropy (selected above) to display the image

#Use this plot to visualise a time series and select the image that corresponds with a point in the time series
def callback(event):
    global time_int, devent
    devent = event
    time_int = event.xdata
    w.value = 'time_int: {}'.format(time_int)

fig = plt.figure(figsize=(6,4))

fig.canvas.mpl_connect('button_press_event', callback)
plt.show()
display(w)
myplot.plot(linestyle= '--', c= 'b', marker = '8', mec = 'b', mfc ='r');
plt.grid()
plt.title(mytitle)

In [ ]:
# Plot image from selected time slice

time_slice = matplotlib.dates.num2date(time_int).date()
myimage.sel(time=time_slice, method='nearest').plot(vmin=0.2, vmax=1.5, figsize=(8,6));
plt.title(time_slice)

# Show Annual Statistics
## Annual difference in RVI, Entropy and NDVI

In [ ]:
# Calculate annual range (max minus min) for radar backscatter RVI, dul-pol decomposition Entropy, and NDVI
from matplotlib import pyplot as plt
pylab.rcParams['font.size']=7

smoothed['RVI_range'] = smoothed.RVI.max(dim='time') - smoothed.RVI.min(dim='time')
dp['entropy_range'] = dp.entropy.max(dim='time') - dp.entropy.min(dim='time')
s2_good_clean['ndvi_range'] = s2_good_clean.ndvi.max(dim='time') - s2_good_clean.ndvi.min(dim='time')

fix, axes = plt.subplots(ncols=3, figsize=(9.5,2.2))

smoothed.RVI_range.plot(vmin=0.2,vmax=1.2, ax=axes[0]);
dp.entropy_range.plot(vmin=0.1,vmax=0.5, ax=axes[1]);
s2_good_clean.ndvi_range.plot(vmin=0.0,vmax=0.5, ax=axes[2]);
plt.tight_layout()
plt.draw()

## Annual RVI, Entropy and NDVI mean, min, max and std

In [ ]:
# Show RVI, Entropy and NDVI mean, min, max and stand deviation for current year

fig, axes = plt.subplots(ncols=4, nrows=3, figsize=(10,5))

smoothed.RVI.mean(dim='time').plot(vmin=0.0,vmax=1.0, ax=axes[0,0]);
smoothed.RVI.min(dim='time').plot(vmin=0.0,vmax=1.0, ax=axes[0,1]);
smoothed.RVI.max(dim='time').plot(vmin=0.0,vmax=2.0, ax=axes[0,2]);
smoothed.RVI.std(dim='time').plot(vmin=0.0,vmax=0.5, ax=axes[0,3]);
dp.entropy.mean(dim='time').plot(vmin=0.2,vmax=0.8, ax=axes[1,0]);
dp.entropy.min(dim='time').plot(vmin=0.0,vmax=0.8, ax=axes[1,1]);
dp.entropy.max(dim='time').plot(vmin=0.0,vmax=1.2, ax=axes[1,2]);
dp.entropy.std(dim='time').plot(vmin=0.05,vmax=0.2, ax=axes[1,3]);
s2_good_clean.ndvi.mean(dim='time').plot(vmin=0.1,vmax=0.5, ax=axes[2,0]);
s2_good_clean.ndvi.min(dim='time').plot(vmin=0.0,vmax=0.5, ax=axes[2,1]);
s2_good_clean.ndvi.max(dim='time').plot(vmin=0.0,vmax=1.0, ax=axes[2,2]);
s2_good_clean.ndvi.std(dim='time').plot(vmin=0.0,vmax=0.2, ax=axes[2,3]);

axes[0,0].set_title('Mean')
axes[0,1].set_title('Min')
axes[0,2].set_title('Max')
axes[0,3].set_title('Std')
plt.tight_layout()
plt.draw()

## Calculate and view monthly mean RVI images

In [ ]:
# plot RVI, Entropy and NDVI through time based on the x,y point selected
#%matplotlib inline
import ipywidgets as widgets
from ipywidgets import interact
from matplotlib import pyplot as plt

import pandas as pd
time = pd.to_datetime(['2017-01-01','2017-02-01','2017-03-01','2017-04-01','2017-05-01','2017-06-01',
                             '2017-07-01','2017-08-01','2017-09-01','2017-10-01','2017-11-01','2017-12-01'])

monthly_mean_RVI=xr.Dataset({'Jan':(smoothed.RVI.sel(time=slice('2017-01-01', '2017-01-31')).mean(dim='time')), 'time': time})
monthly_mean_RVI['Feb']=smoothed.RVI.sel(time=slice('2017-02-01', '2017-02-28')).mean(dim='time')
monthly_mean_RVI['Mar']=smoothed.RVI.sel(time=slice('2017-03-01', '2017-03-31')).mean(dim='time')
monthly_mean_RVI['Apr']=smoothed.RVI.sel(time=slice('2017-04-01', '2017-04-30')).mean(dim='time')
monthly_mean_RVI['May']=smoothed.RVI.sel(time=slice('2017-05-01', '2017-05-31')).mean(dim='time')
monthly_mean_RVI['Jun']=smoothed.RVI.sel(time=slice('2017-06-01', '2017-06-30')).mean(dim='time')
monthly_mean_RVI['Jul']=smoothed.RVI.sel(time=slice('2017-07-01', '2017-07-31')).mean(dim='time')
monthly_mean_RVI['Aug']=smoothed.RVI.sel(time=slice('2017-08-01', '2017-08-31')).mean(dim='time')
monthly_mean_RVI['Sep']=smoothed.RVI.sel(time=slice('2017-09-01', '2017-09-30')).mean(dim='time')
monthly_mean_RVI['Oct']=smoothed.RVI.sel(time=slice('2017-10-01', '2017-10-31')).mean(dim='time')
monthly_mean_RVI['Nov']=smoothed.RVI.sel(time=slice('2017-11-01', '2017-11-30')).mean(dim='time')
monthly_mean_RVI['Dec']=smoothed.RVI.sel(time=slice('2017-12-01', '2017-12-31')).mean(dim='time')

#monthly_mean_RVI = monthly_mean_RVI.assign_attrs(bs_attrs)

fig = plt.figure()

@interact(month=['Jan','Feb','Mar','Apr','May','Jun','Jul','Aug','Sep','Oct','Nov','Dec'])

def plot(month='Jan'):
    if month == 'Jan': plt.imshow(np.flip(monthly_mean_RVI.Jan), clim=(0,1.0))
    if month == 'Feb': plt.imshow(np.flip(monthly_mean_RVI.Feb), clim=(0,1.0))
    if month == 'Mar': plt.imshow(np.flip(monthly_mean_RVI.Mar), clim=(0,1.0))
    if month == 'Apr': plt.imshow(np.flip(monthly_mean_RVI.Apr), clim=(0,1.0))
    if month == 'May': plt.imshow(np.flip(monthly_mean_RVI.May), clim=(0,1.0))
    if month == 'Jun': plt.imshow(np.flip(monthly_mean_RVI.Jun), clim=(0,1.0))
    if month == 'Jul': plt.imshow(np.flip(monthly_mean_RVI.Jul), clim=(0,1.0))
    if month == 'Aug': plt.imshow(np.flip(monthly_mean_RVI.Aug), clim=(0,1.0))
    if month == 'Sep': plt.imshow(np.flip(monthly_mean_RVI.Sep), clim=(0,1.0))
    if month == 'Oct': plt.imshow(np.flip(monthly_mean_RVI.Oct), clim=(0,1.0))
    if month == 'Nov': plt.imshow(np.flip(monthly_mean_RVI.Nov), clim=(0,1.0))
    if month == 'Dec': plt.imshow(np.flip(monthly_mean_RVI.Dec), clim=(0,1.0))


## Calculate and view monthly mean Entropy images

In [ ]:
# Display a monthly mean
# plot RVI, Entropy and NDVI through time based on the x,y point selected
#%matplotlib inline
import ipywidgets as widgets
from ipywidgets import interact
from matplotlib import pyplot as plt

import pandas as pd
time = pd.to_datetime(['2017-01-01','2017-02-01','2017-03-01','2017-04-01','2017-05-01','2017-06-01',
                             '2017-07-01','2017-08-01','2017-09-01','2017-10-01','2017-11-01','2017-12-01'])

monthly_mean_entropy=xr.Dataset({'Jan':(dp.entropy.sel(time=slice('2017-01-01', '2017-01-31')).mean(dim='time')), 'time': time})
monthly_mean_entropy['Feb']=dp.entropy.sel(time=slice('2017-02-01', '2017-02-28')).mean(dim='time')
monthly_mean_entropy['Mar']=dp.entropy.sel(time=slice('2017-03-01', '2017-03-31')).mean(dim='time')
monthly_mean_entropy['Apr']=dp.entropy.sel(time=slice('2017-04-01', '2017-04-30')).mean(dim='time')
monthly_mean_entropy['May']=dp.entropy.sel(time=slice('2017-05-01', '2017-05-31')).mean(dim='time')
monthly_mean_entropy['Jun']=dp.entropy.sel(time=slice('2017-06-01', '2017-06-30')).mean(dim='time')
monthly_mean_entropy['Jul']=dp.entropy.sel(time=slice('2017-07-01', '2017-07-31')).mean(dim='time')
monthly_mean_entropy['Aug']=dp.entropy.sel(time=slice('2017-08-01', '2017-08-31')).mean(dim='time')
monthly_mean_entropy['Sep']=dp.entropy.sel(time=slice('2017-09-01', '2017-09-30')).mean(dim='time')
monthly_mean_entropy['Oct']=dp.entropy.sel(time=slice('2017-10-01', '2017-10-31')).mean(dim='time')
monthly_mean_entropy['Nov']=dp.entropy.sel(time=slice('2017-11-01', '2017-11-30')).mean(dim='time')
monthly_mean_entropy['Dec']=dp.entropy.sel(time=slice('2017-12-01', '2017-12-31')).mean(dim='time')

fig = plt.figure()

@interact(month=['Jan','Feb','Mar','Apr','May','Jun','Jul','Aug','Sep','Oct','Nov','Dec'])

def plot(month='Jan'):
    if month == 'Jan': plt.imshow(np.flip(monthly_mean_entropy.Jan), clim=(0,1.0))
    if month == 'Feb': plt.imshow(np.flip(monthly_mean_entropy.Feb), clim=(0,1.0))
    if month == 'Mar': plt.imshow(np.flip(monthly_mean_entropy.Mar), clim=(0,1.0))
    if month == 'Apr': plt.imshow(np.flip(monthly_mean_entropy.Apr), clim=(0,1.0))
    if month == 'May': plt.imshow(np.flip(monthly_mean_entropy.May), clim=(0,1.0))
    if month == 'Jun': plt.imshow(np.flip(monthly_mean_entropy.Jun), clim=(0,1.0))
    if month == 'Jul': plt.imshow(np.flip(monthly_mean_entropy.Jul), clim=(0,1.0))
    if month == 'Aug': plt.imshow(np.flip(monthly_mean_entropy.Aug), clim=(0,1.0))
    if month == 'Sep': plt.imshow(np.flip(monthly_mean_entropy.Sep), clim=(0,1.0))
    if month == 'Oct': plt.imshow(np.flip(monthly_mean_entropy.Oct), clim=(0,1.0))
    if month == 'Nov': plt.imshow(np.flip(monthly_mean_entropy.Nov), clim=(0,1.0))
    if month == 'Dec': plt.imshow(np.flip(monthly_mean_entropy.Dec), clim=(0,1.0))

# Write output scenes to GeoTIFFs
## Output annual range and monthly mean RVI and Entropy images to GeoTIFF format

In [ ]:
@interact(myoutputdir=['/g/data/qd04/Cate/TempProcessing/','/g/data/qd04/Cate/'])

def plot(myoutputdir='/g/data/qd04/Cate/TempProcessing/'):
    global mydir
    if myoutputdir == '/g/data/qd04/Cate/TempProcessing/': mydir = '/g/data/qd04/Cate/TempProcessing/'
    if myoutputdir == '/g/data/qd04/Cate/':mydir = '/g/data/qd04/Cate/'

In [ ]:
@interact(myoutputfiles=['output files','stop'])

def plot(myoutputfiles='stop'):
    if myoutputfiles == 'output files': 
        print('Writing GeoTIFF files')
    
        # write output images (in Albers) to GeoTIFF

        import ogr, gdal, osr

        #mydir = '/g/data/qd04/Cate/TempProcessing/'

        # define coordinates for Albers equal area (3577)
        xcoords = smoothed.isel(time=1).vv.indexes['x']
        ycoords = smoothed.isel(time=1).vv.indexes['y']
        yt,xt = smoothed.isel(time=1).vv.shape
        MaxValX = xcoords.shape
        MaxValY = ycoords.shape

        # set geotransform and output projection
        xres = 25 
        yres = 25 
        geotransform = (xcoords[MaxValX[0]-1]-(xres*0.5), xres, 0, ycoords[MaxValY[0]-1]+(yres*0.5), 0, -yres) # offset by half the pixel size since it needs to be top-left pixel coord
        srs = osr.SpatialReference() 
        srs.ImportFromEPSG(3577)

        # loop through monthly images
        months = ['Jan','Feb','Mar','Apr','May','Jun','Jul','Aug','Sep','Oct','Nov','Dec']

        # Edit output filenames (filename_RVI and filename_entropy) and directory as required
        for month in months:

            filename_RVI = 'S1_RVI_3577_'+ month +'_'+ Year +'.tif' 
            S1_ds = gdal.GetDriverByName('GTiff').Create(mydir+filename_RVI, 
                                                           xt, yt, 1, gdal.GDT_Float32)
            S1_ds.SetGeoTransform(geotransform) # specify coordinates
            S1_ds.SetProjection(srs.ExportToWkt()) # export coords to file
            S1_ds.GetRasterBand(1).WriteArray(np.flip(monthly_mean_RVI[month]).data) # write band to raster
            S1_ds.FlushCache()  # write to file
            S1_ds = None # save and close

            filename_entropy = 'S1_Entropy_3577_'+ month + '_'+ Year + '.tif' 
            S1_ds = gdal.GetDriverByName('GTiff').Create(mydir+filename_entropy, 
                                                           xt, yt, 1, gdal.GDT_Float32)
            S1_ds.SetGeoTransform(geotransform) # specify coordinates
            S1_ds.SetProjection(srs.ExportToWkt()) # export coords to file
            S1_ds.GetRasterBand(1).WriteArray(np.flip(monthly_mean_entropy[month]).data) # write band to raster
            S1_ds.FlushCache()  # write to file
            S1_ds = None # save and close

        # Output annual RVI and Entropy range images
        filename_RVI = 'S1_RVI_3577_Annual_Range_' + Year +'.tif' 
        S1_ds = gdal.GetDriverByName('GTiff').Create(mydir+filename_RVI, 
                                                           xt, yt, 1, gdal.GDT_Float32)
        S1_ds.SetGeoTransform(geotransform) # specify coordinates
        S1_ds.SetProjection(srs.ExportToWkt()) # export coords to file
        S1_ds.GetRasterBand(1).WriteArray(np.flip(smoothed.RVI_range).data) # write band to raster
        S1_ds.FlushCache()  # write to file
        S1_ds = None # save and close

        # Output annual RVI and Entropy range images
        filename_entropy = 'S1_Entropy_3577_Annual_Range_' + Year + '.tif' 
        S1_ds = gdal.GetDriverByName('GTiff').Create(mydir+filename_entropy, 
                                                           xt, yt, 1, gdal.GDT_Float32)
        S1_ds.SetGeoTransform(geotransform) # specify coordinates
        S1_ds.SetProjection(srs.ExportToWkt()) # export coords to file
        S1_ds.GetRasterBand(1).WriteArray(np.flip(dp.entropy_range).data) # write band to raster
        S1_ds.FlushCache()  # write to file
        S1_ds = None # save and close

    if myoutputfiles == 'stop': print('Finished without writing GeoTIFF files')

In [ ]:
client.close()
client = None

In [ ]:
cluster.close()
cluster = None